# Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Utils functions

In [ ]:
def get_centroid(x1,y1,x2,y2):
  c1 = float((x2 + x1) / 2.0)
  c2 = float((y2 + y1) / 2.0)
  return (c1, c2)

In [ ]:
import math

def euclidean_distance(location1, location2, shape_reference):
  image_hight, image_width, _ = shape_reference
  return math.sqrt(math.pow(math.floor(location2.x * image_width) - math.floor(location1.y * image_hight), 2) + math.pow(math.floor(location2.x * image_width) - math.floor(location1.y * image_hight), 2))

def euclidean_distance_dict(location1, location2, shape_reference):
  image_hight, image_width, _ = shape_reference
  return math.sqrt(math.pow(math.floor(location2['x']) - math.floor(location1['y']), 2) + math.pow(math.floor(location2['x']) - math.floor(location1['y']), 2))

# Install mediapipe

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.21 which is incompatible.


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from google.colab.patches import cv2_imshow

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2_imshow(img)

In [ ]:
def __rotate(origin: tuple, point: tuple, angle: float):
    """
    Rotates a point counterclockwise by a given angle around a given origin.

    :param origin: Landmark in the (X, Y) format of the origin from which to count angle of rotation
    :param point: Landmark in the (X, Y) format to be rotated
    :param angle: Angle under which the point shall be rotated
    :return: New landmarks (coordinates)
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)

    return (math.floor(qx), math.floor(qy))

In [ ]:
import random

def augment_arm_joint_rotate(arm_postures: dict, probability: float, angle_range: tuple) -> dict:
    """
    AUGMENTATION TECHNIQUE. The joint coordinates of both arms are passed successively, and the impending landmark is
    slightly rotated with respect to the current one. The chance of each joint to be rotated is 3:10 and the angle of
    alternation is a uniform random angle up to +-4 degrees. This simulates slight, negligible variances in each
    execution of a sign, which do not change its semantic meaning.

    :param sign: Dictionary with sequential skeletal data of the signing person
    :param probability: Probability of each joint to be rotated (float from the range [0, 1])
    :param angle_range: Tuple containing the angle range (minimal and maximal angle in degrees) to randomly choose the
                        angle by which the landmarks will be rotated from

    :return: Dictionary with augmented (by arm joint rotation) sequential skeletal data of the signing person
    """
    ARM_IDENTIFIERS_ORDER = ["$side$_SHOULDER", "$side$_ELBOW", "$side$_WRIST"]

    # Iterate gradually over the landmarks on arm

    for side in ['LEFT','RIGHT']:
      for landmark_index, landmark_origin in enumerate(ARM_IDENTIFIERS_ORDER):
          landmark_origin = landmark_origin.replace("$side$", side)

          # End the process on the current hand if the landmark is not present
          if landmark_origin not in arm_postures:
              break

          # Perform rotation by provided probability
              
          if random.random() < probability:
              angle = math.radians(random.uniform(*angle_range))

              for to_be_rotated in ARM_IDENTIFIERS_ORDER[landmark_index + 1:]:
                  to_be_rotated = to_be_rotated.replace("$side$", side)

                  # Skip if the landmark is not present
                  if to_be_rotated not in arm_postures:
                      continue

                  arm_postures[to_be_rotated] = __rotate(arm_postures[landmark_origin], arm_postures[to_be_rotated], angle)

    return arm_postures

In [ ]:
def augment_rotate(arm_postures: dict, hand_landmarks: dict, angle_range: tuple) -> dict:
    """
    AUGMENTATION TECHNIQUE. All the joint coordinates in each frame are rotated by a random angle up to 13 degrees with
    the center of rotation lying in the center of the frame, which is equal to [0.5; 0.5].

    :param sign: Dictionary with sequential skeletal data of the signing person
    :param angle_range: Tuple containing the angle range (minimal and maximal angle in degrees) to randomly choose the
                        angle by which the landmarks will be rotated from

    :return: Dictionary with augmented (by rotation) sequential skeletal data of the signing person
    """

    angle = math.radians(random.uniform(*angle_range))

    for key, value in arm_postures.items():
      arm_postures[key] = __rotate((0.5, 0.5), value, angle)
    for key, value in hand_landmarks.items():
      hand_landmarks[key]['x'], hand_landmarks[key]['y'] = __rotate((0.5, 0.5), (value['x'], value['y']), angle)

    return  arm_postures, hand_landmarks

In [ ]:
def hands_distances(results, hands_landmarks, frame):
    hkpts_list = []
    
    hkp_reference = -1
    if results.left_hand_landmarks:
      if hands_landmarks['LEFT_PINKY_TIP']['y'] <= hands_landmarks['LEFT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_PINKY_TIP']['y'] <= hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_PINKY_TIP']['y'] <= hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_PINKY_TIP']['y'] <= hands_landmarks['LEFT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'],hands_landmarks['LEFT_PINKY_TIP'],frame.shape)
      elif hands_landmarks['LEFT_RING_FINGER_TIP']['y'] <= hands_landmarks['LEFT_PINKY_TIP']['y'] \
        and hands_landmarks['LEFT_RING_FINGER_TIP']['y'] <= hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_RING_FINGER_TIP']['y'] <= hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_RING_FINGER_TIP']['y'] <= hands_landmarks['LEFT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'],hands_landmarks['LEFT_RING_FINGER_TIP'],frame.shape)
      elif hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['LEFT_PINKY_TIP']['y'] \
        and hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['LEFT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['LEFT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'],hands_landmarks['LEFT_MIDDLE_FINGER_TIP'],frame.shape)
      elif hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['LEFT_PINKY_TIP']['y'] \
        and hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['LEFT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['LEFT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'],hands_landmarks['LEFT_INDEX_FINGER_TIP'],frame.shape)
      elif hands_landmarks['LEFT_THUMB_TIP']['y'] <= hands_landmarks['LEFT_PINKY_TIP']['y'] \
        and hands_landmarks['LEFT_THUMB_TIP']['y'] <= hands_landmarks['LEFT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_THUMB_TIP']['y'] <= hands_landmarks['LEFT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['LEFT_THUMB_TIP']['y'] <= hands_landmarks['LEFT_INDEX_FINGER_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'],hands_landmarks['LEFT_THUMB_TIP'],frame.shape)
          
      hkp1 = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'], hands_landmarks['LEFT_MIDDLE_FINGER_MCP'],frame.shape)
      hkp1 = hkp1 / hkp_reference if hkp_reference > 0 else hkp1
      hkpts_list.append(hkp1) #0/9
      hkp2 = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'], hands_landmarks['LEFT_THUMB_MCP'],frame.shape)
      hkp2 = hkp2 / hkp_reference if hkp_reference > 0 else hkp2
      hkpts_list.append(hkp2) #0/2
      hkp3 = euclidean_distance_dict(hands_landmarks['LEFT_WRIST'], hands_landmarks['LEFT_PINKY_MCP'],frame.shape)
      hkp3 = hkp3 / hkp_reference if hkp_reference > 0 else hkp3
      hkpts_list.append(hkp3) #0/17
      hkp4 = euclidean_distance_dict(hands_landmarks['LEFT_PINKY_MCP'], hands_landmarks['LEFT_PINKY_TIP'],frame.shape)
      hkp4 = hkp4 / hkp_reference if hkp_reference > 0 else hkp4
      hkpts_list.append(hkp4) #17/20
      hkp5 = euclidean_distance_dict(hands_landmarks['LEFT_RING_FINGER_MCP'], hands_landmarks['LEFT_RING_FINGER_TIP'] ,frame.shape)
      hkp5 = hkp5 / hkp_reference if hkp_reference > 0 else hkp5
      hkpts_list.append(hkp5) #13/16
      hkp6 = euclidean_distance_dict(hands_landmarks['LEFT_MIDDLE_FINGER_MCP'], hands_landmarks['LEFT_MIDDLE_FINGER_TIP'],frame.shape)
      hkp6 = hkp6 / hkp_reference if hkp_reference > 0 else hkp6
      hkpts_list.append(hkp6) #9/12
      hkp7 = euclidean_distance_dict(hands_landmarks['LEFT_INDEX_FINGER_MCP'], hands_landmarks['LEFT_INDEX_FINGER_TIP'],frame.shape)
      hkp7 = hkp7 / hkp_reference if hkp_reference > 0 else hkp7
      hkpts_list.append(hkp7) #5/8
      hkp8 = euclidean_distance_dict(hands_landmarks['LEFT_THUMB_MCP'], hands_landmarks['LEFT_THUMB_TIP'],frame.shape)
      hkp8 = hkp8 / hkp_reference if hkp_reference > 0 else hkp8
      hkpts_list.append(hkp8) #2/4
    else:
      hkpts_list.append(0) #0/9
      hkpts_list.append(0) #0/2
      hkpts_list.append(0) #0/17
      hkpts_list.append(0) #17/20
      hkpts_list.append(0) #13/16
      hkpts_list.append(0) #9/12
      hkpts_list.append(0) #5/8
      hkpts_list.append(0) #2/4

    hkp_reference = -1
    if results.right_hand_landmarks:
      if hands_landmarks['RIGHT_PINKY_TIP']['y'] <= hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_PINKY_TIP']['y'] <= hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_PINKY_TIP']['y'] <= hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_PINKY_TIP']['y'] <= hands_landmarks['RIGHT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'],hands_landmarks['RIGHT_PINKY_TIP'],frame.shape)
      elif hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_PINKY_TIP']['y'] \
        and hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'],hands_landmarks['RIGHT_RING_FINGER_TIP'],frame.shape)
      elif hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_PINKY_TIP']['y'] \
        and hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'],hands_landmarks['RIGHT_MIDDLE_FINGER_TIP'],frame.shape)
      elif hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_PINKY_TIP']['y'] \
        and hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y'] <= hands_landmarks['RIGHT_THUMB_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'],hands_landmarks['RIGHT_INDEX_FINGER_TIP'],frame.shape)
      elif hands_landmarks['RIGHT_THUMB_TIP']['y'] <= hands_landmarks['RIGHT_PINKY_TIP']['y'] \
        and hands_landmarks['RIGHT_THUMB_TIP']['y'] <= hands_landmarks['RIGHT_RING_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_THUMB_TIP']['y'] <= hands_landmarks['RIGHT_MIDDLE_FINGER_TIP']['y'] \
        and hands_landmarks['RIGHT_THUMB_TIP']['y'] <= hands_landmarks['RIGHT_INDEX_FINGER_TIP']['y']:
        hkp_reference = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'],hands_landmarks['RIGHT_THUMB_TIP'],frame.shape)
          
      hkp1 = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'], hands_landmarks['RIGHT_MIDDLE_FINGER_MCP'],frame.shape)
      hkp1 = hkp1 / hkp_reference if hkp_reference > 0 else hkp1
      hkpts_list.append(hkp1) #0/9
      hkp2 = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'], hands_landmarks['RIGHT_THUMB_MCP'],frame.shape)
      hkp2 = hkp2 / hkp_reference if hkp_reference > 0 else hkp2
      hkpts_list.append(hkp2) #0/2
      hkp3 = euclidean_distance_dict(hands_landmarks['RIGHT_WRIST'], hands_landmarks['RIGHT_PINKY_MCP'],frame.shape)
      hkp3 = hkp3 / hkp_reference if hkp_reference > 0 else hkp3
      hkpts_list.append(hkp3) #0/17
      hkp4 = euclidean_distance_dict(hands_landmarks['RIGHT_PINKY_MCP'], hands_landmarks['RIGHT_PINKY_TIP'],frame.shape)
      hkp4 = hkp4 / hkp_reference if hkp_reference > 0 else hkp4
      hkpts_list.append(hkp4) #17/20
      hkp5 = euclidean_distance_dict(hands_landmarks['RIGHT_RING_FINGER_MCP'], hands_landmarks['RIGHT_RING_FINGER_TIP'] ,frame.shape)
      hkp5 = hkp5 / hkp_reference if hkp_reference > 0 else hkp5
      hkpts_list.append(hkp5) #13/16
      hkp6 = euclidean_distance_dict(hands_landmarks['RIGHT_MIDDLE_FINGER_MCP'], hands_landmarks['RIGHT_MIDDLE_FINGER_TIP'],frame.shape)
      hkp6 = hkp6 / hkp_reference if hkp_reference > 0 else hkp6
      hkpts_list.append(hkp6) #9/12
      hkp7 = euclidean_distance_dict(hands_landmarks['RIGHT_INDEX_FINGER_MCP'], hands_landmarks['RIGHT_INDEX_FINGER_TIP'],frame.shape)
      hkp7 = hkp7 / hkp_reference if hkp_reference > 0 else hkp7
      hkpts_list.append(hkp7) #5/8
      hkp8 = euclidean_distance_dict(hands_landmarks['RIGHT_THUMB_MCP'], hands_landmarks['RIGHT_THUMB_TIP'],frame.shape)
      hkp8 = hkp8 / hkp_reference if hkp_reference > 0 else hkp8
      hkpts_list.append(hkp8) #2/4
    else:
      hkpts_list.append(0) #0/9
      hkpts_list.append(0) #0/2
      hkpts_list.append(0) #0/17
      hkpts_list.append(0) #17/20
      hkpts_list.append(0) #13/16
      hkpts_list.append(0) #9/12
      hkpts_list.append(0) #5/8
      hkpts_list.append(0) #2/4

    return hkpts_list

# Pose and body estimation

In [ ]:
import mediapipe as mp
import math
from random import randrange
import pandas as pd

str_video = '01067'

mp_holistic = mp.solutions.holistic

input_movie = cv2.VideoCapture('/content/gdrive/My Drive/your_directory/LIBRAS Data/' + str_video + '.mp4')
frame_number = 1
FRAME_WINDOW = 3
AUGMENTATIONS_PROB = 1

cols_list = []
rows_list = []
data_rows_list = []

while True:
  ret, frame = input_movie.read()
  
  if not ret:
    break

  if frame_number % FRAME_WINDOW == 0:
    
    with mp_holistic.Holistic(
      static_image_mode=False, min_detection_confidence=0.6, model_complexity=2) as holistic:

      # Convert the BGR image to RGB and process it with MediaPipe Pose.
      results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

      image_hight, image_width, _ = frame.shape
      
      if results.pose_landmarks:
        '''
          Arm Postures
        '''
        
        pkp1 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x * image_width) # x coordinate kp 1
        pkp2 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y * image_hight) # x coordinate kp 2
        pkp3 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x * image_width) # x coordinate kp 3
        pkp4 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y * image_hight) # y coordinate kp 4
        pkp5 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x * image_width) # x coordinate kp 5
        pkp6 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y * image_hight) # y coordinate kp 6
        pkp7 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x * image_width) # x coordinate kp 7
        pkp8 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y * image_hight) # x coordinate kp 8
        pkp9 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x * image_width) # x coordinate kp 9
        pkp10 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y * image_hight) # y coordinate kp 10
        pkp11 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x * image_width) # x coordinate kp 11
        pkp12 = math.floor(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y * image_hight) # y coordinate kp 12
        arm_postures_dic = {'RIGHT_SHOULDER':(pkp1, pkp2),
                            'RIGHT_ELBOW':(pkp3,pkp4),
                            'RIGHT_WRIST':(pkp5,pkp6),
                            'LEFT_SHOULDER':(pkp7,pkp8),
                            'LEFT_ELBOW':(pkp9,pkp10),
                            'LEFT_WRIST':(pkp11,pkp12)}

      if results.face_landmarks:  
        '''
          Facial Expressions
        '''
        #https://github.com/google/mediapipe/blob/a908d668c730da128dfa8d9f6bd25d519d006692/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png esto es para ver los indices del rostro
        fkp_width_reference = euclidean_distance(results.face_landmarks.landmark[234], results.face_landmarks.landmark[454], frame.shape)
        fkp_height_reference = euclidean_distance(results.face_landmarks.landmark[10], results.face_landmarks.landmark[152], frame.shape)
        fkp1 = euclidean_distance(results.face_landmarks.landmark[0], results.face_landmarks.landmark[17], frame.shape)
        fkp1 = fkp1 / fkp_height_reference if fkp_height_reference > 0 else fkp1
        fkp2 = euclidean_distance(results.face_landmarks.landmark[61], results.face_landmarks.landmark[291], frame.shape)
        fkp2 = fkp2 / fkp_width_reference if fkp_width_reference > 0 else fkp2
        fkp3 = euclidean_distance(results.face_landmarks.landmark[0], results.face_landmarks.landmark[94], frame.shape)
        fkp3 = fkp3 / fkp_height_reference if fkp_height_reference > 0 else fkp3
        fkp4 = euclidean_distance(results.face_landmarks.landmark[52], results.face_landmarks.landmark[159], frame.shape)
        fkp4 = fkp4 / fkp_height_reference if fkp_height_reference > 0 else fkp4
        fkp5 = euclidean_distance(results.face_landmarks.landmark[282], results.face_landmarks.landmark[386], frame.shape)
        fkp5 = fkp5 / fkp_height_reference if fkp_height_reference > 0 else fkp5

      if results.left_hand_landmarks and results.right_hand_landmarks:
        hands_landmarks = {'LEFT_PINKY_TIP': {'x':math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].x * image_width), 
                                              'y': math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].y * image_hight)},
                           'LEFT_RING_FINGER_TIP': {'x':math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].x * image_width), 
                                                    'y': math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].y * image_hight)},
                           'LEFT_MIDDLE_FINGER_TIP' : {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].x * image_width),
                                                       'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].y * image_hight)},
                           'LEFT_INDEX_FINGER_TIP' : {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x * image_width),
                                                      'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y * image_hight)},
                           'LEFT_THUMB_TIP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].x * image_width),
                                              'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].y * image_hight)},
                           'LEFT_WRIST': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image_width),
                                          'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image_hight)},
                           'LEFT_MIDDLE_FINGER_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].x * image_width),
                                                      'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].y * image_hight)},
                           'LEFT_THUMB_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].x * image_width),
                                              'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].y * image_hight)},
                           'LEFT_PINKY_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].x * image_width),
                                              'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].y * image_hight)},
                           'LEFT_RING_FINGER_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].x * image_width),
                                                    'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].y * image_hight)},
                           'LEFT_INDEX_FINGER_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].x * image_width),
                                                     'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].y * image_hight)},
                           'RIGHT_PINKY_TIP': {'x':math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].x * image_width), 
                                              'y': math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].y * image_hight)},
                           'RIGHT_RING_FINGER_TIP': {'x':math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].x * image_width), 
                                                    'y': math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].y * image_hight)},
                           'RIGHT_MIDDLE_FINGER_TIP' : {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].x * image_width),
                                                       'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].y * image_hight)},
                           'RIGHT_INDEX_FINGER_TIP' : {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x * image_width),
                                                      'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y * image_hight)},
                           'RIGHT_THUMB_TIP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].x * image_width),
                                              'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].y * image_hight)},
                           'RIGHT_WRIST': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image_width),
                                          'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image_hight)},
                           'RIGHT_MIDDLE_FINGER_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].x * image_width),
                                                      'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].y * image_hight)},
                           'RIGHT_THUMB_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].x * image_width),
                                              'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].y * image_hight)},
                           'RIGHT_PINKY_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].x * image_width),
                                              'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].y * image_hight)},
                           'RIGHT_RING_FINGER_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].x * image_width),
                                                    'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].y * image_hight)},
                           'RIGHT_INDEX_FINGER_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].x * image_width),
                                                     'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].y * image_hight)}}
        hkpts_list = hands_distances(results, hands_landmarks, frame)
      elif results.left_hand_landmarks:
        hands_landmarks = {'LEFT_PINKY_TIP': {'x':math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].x * image_width), 
                                              'y': math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].y * image_hight)},
                           'LEFT_RING_FINGER_TIP': {'x':math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].x * image_width), 
                                                    'y': math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].y * image_hight)},
                           'LEFT_MIDDLE_FINGER_TIP' : {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].x * image_width),
                                                       'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].y * image_hight)},
                           'LEFT_INDEX_FINGER_TIP' : {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x * image_width),
                                                      'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y * image_hight)},
                           'LEFT_THUMB_TIP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].x * image_width),
                                              'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].y * image_hight)},
                           'LEFT_WRIST': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image_width),
                                          'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image_hight)},
                           'LEFT_MIDDLE_FINGER_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].x * image_width),
                                                      'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].y * image_hight)},
                           'LEFT_THUMB_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].x * image_width),
                                              'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].y * image_hight)},
                           'LEFT_PINKY_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].x * image_width),
                                              'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].y * image_hight)},
                           'LEFT_RING_FINGER_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].x * image_width),
                                                    'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].y * image_hight)},
                           'LEFT_INDEX_FINGER_MCP': {'x' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].x * image_width),
                                                     'y' : math.floor(results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].y * image_hight)}}
        hkpts_list = hands_distances(results, hands_landmarks, frame)
      elif results.right_hand_landmarks:
        hands_landmarks = {'RIGHT_PINKY_TIP': {'x':math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].x * image_width), 
                                              'y': math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP].y * image_hight)},
                           'RIGHT_RING_FINGER_TIP': {'x':math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].x * image_width), 
                                                    'y': math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_TIP].y * image_hight)},
                           'RIGHT_MIDDLE_FINGER_TIP' : {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].x * image_width),
                                                       'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP].y * image_hight)},
                           'RIGHT_INDEX_FINGER_TIP' : {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x * image_width),
                                                      'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y * image_hight)},
                           'RIGHT_THUMB_TIP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].x * image_width),
                                              'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP].y * image_hight)},
                           'RIGHT_WRIST': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image_width),
                                          'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image_hight)},
                           'RIGHT_MIDDLE_FINGER_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].x * image_width),
                                                      'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].y * image_hight)},
                           'RIGHT_THUMB_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].x * image_width),
                                              'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_MCP].y * image_hight)},
                           'RIGHT_PINKY_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].x * image_width),
                                              'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_MCP].y * image_hight)},
                           'RIGHT_RING_FINGER_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].x * image_width),
                                                    'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.RING_FINGER_MCP].y * image_hight)},
                           'RIGHT_INDEX_FINGER_MCP': {'x' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].x * image_width),
                                                     'y' : math.floor(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_MCP].y * image_hight)}}
        hkpts_list = hands_distances(results, hands_landmarks, frame)
      else:

        hands_landmarks = None
        hkpts_list = [0 for i in range(0,16)]
      
      #Generating the feature vector
      data = [frame_number,pkp1,pkp2,pkp3,pkp4,pkp5,pkp6,pkp7,pkp8,pkp9,pkp10,pkp11,pkp12,fkp1,fkp2,fkp3,fkp4,fkp5]
      #print(data)
      for index in hkpts_list: 
        data.append(index)
      data_rows_list.append(data)

      rdm = random.random()
      if rdm < AUGMENTATIONS_PROB:
        selected_aug = randrange(2)
        selected_aug = 1

        #if hands_landmarks != None:
        if selected_aug == 0 and hands_landmarks != None:
          out_aug, hands_landmarks = augment_rotate(arm_postures_dic, hands_landmarks, (-13, 13))
          data = [frame_number, out_aug['RIGHT_SHOULDER'][0],out_aug['RIGHT_SHOULDER'][1],out_aug['RIGHT_ELBOW'][0],out_aug['RIGHT_ELBOW'][1],out_aug['RIGHT_WRIST'][0],out_aug['RIGHT_WRIST'][1],
                  out_aug['LEFT_SHOULDER'][0],out_aug['LEFT_SHOULDER'][1],out_aug['LEFT_ELBOW'][0],out_aug['LEFT_ELBOW'][1],out_aug['LEFT_WRIST'][0],out_aug['LEFT_WRIST'][1]]
          data.append(fkp1)
          data.append(fkp2)
          data.append(fkp3)
          data.append(fkp4)
          data.append(fkp5)
          hkpts_list_aug = hands_distances(results, hands_landmarks, frame)
          for index in hkpts_list_aug: 
            data.append(index)
          data_rows_list.append(data)
        
        if selected_aug == 1:
          original_postures = arm_postures_dic.copy()
          out_aug = augment_arm_joint_rotate(arm_postures_dic, 1, (-4, 4))
          data = [frame_number, out_aug['RIGHT_SHOULDER'][0],out_aug['RIGHT_SHOULDER'][1],out_aug['RIGHT_ELBOW'][0],out_aug['RIGHT_ELBOW'][1],out_aug['RIGHT_WRIST'][0],out_aug['RIGHT_WRIST'][1],
                  out_aug['LEFT_SHOULDER'][0],out_aug['LEFT_SHOULDER'][1],out_aug['LEFT_ELBOW'][0],out_aug['LEFT_ELBOW'][1],out_aug['LEFT_WRIST'][0],out_aug['LEFT_WRIST'][1]]
          data.append(fkp1)
          data.append(fkp2)
          data.append(fkp3)
          data.append(fkp4)
          data.append(fkp5)
          for index in hkpts_list: 
            data.append(index)
          data_rows_list.append(data)
      
  frame_number += 1

for idx in range(len(data_rows_list)):
  rows_list.append('row_'+str(idx))

for idx in range(len(data_rows_list[0])):
  cols_list.append('col_'+str(idx))

df1 = pd.DataFrame(data_rows_list,
                   index=rows_list)
df1.to_excel("/content/gdrive/My Drive/your_directory/LIBRAS Data/"+ str_video +"_faceposture_output_07102022_1aug.xlsx")